In [21]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import os
import sys
from ocp_table_tpot.globals import Globals as gd
from tpot import TPOTRegressor
sys.path.insert(0,'..')
from src.models.model import HistoricalMedian,XGBoost,LinearModel,RF,KNN,SVM,mase,TimeSeriesSplitImproved
from src.data.split_all_data import DROPCOLS_DIFF_FINAL,DROPCOLS_FINAL
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,MinMaxScaler,Normalizer,StandardScaler,QuantileTransformer
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split,cross_val_predict
from skgarden.quantile import RandomForestQuantileRegressor
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.preprocessing import FunctionTransformer,PolynomialFeatures
from sklearn.svm import SVR
from copy import copy
from tpot.builtins import StackingEstimator
from lightgbm import LGBMRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import make_pipeline, make_union
from catboost import CatBoostRegressor,Pool,cv


df_tsfresh = pd.read_pickle(f'../data/processed/train_test_tsfresh_6.pkl').reset_index(level = 0)
data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')


In [23]:
year = 2019
tgt = 'rougher.output.recovery'
#cols_drop = X.columns[X.columns.str.contains('p24')]
X = data_dict[year][f'X_train_tsclean']#.drop(cols_drop,axis = 1)
y = data_dict[year]['y_train']
mask = data_dict[year]['mask']

X =X.loc[mask.index,:][mask]
y = y.loc[mask.index,:][mask]
print(f'2)  train: {X.shape}')
#X = X.sample(frac=0.4,random_state=123).sort_index().dropna().astype(float)
y= y.loc[X.index,tgt]
X_filt = X.filter(regex  ="rougher|dayw|hour",axis = 1)
print(f'after sample() train: {X.shape}')
#print(f'after sample() filt train: {X_filt.shape}')
# Nmonths_total = 8
# Nspl = int(Nmonths_total * 30 / 15)
# Nmonths_test = 4
# Nmonths_min_train = 2.5
# train_splits = Nspl // Nmonths_total*Nmonths_min_train
# test_splits=int(Nmonths_test / Nmonths_total*Nspl)
# cv = TimeSeriesSplitImproved(n_splits=Nspl)

2)  train: (13590, 388)
after sample() train: (13590, 388)


In [24]:
set(X.columns)

{'daily_avg_final',
 'daily_avg_rougher',
 'dayw',
 'deriv1_encod_dif_rougher.input.feed_fe',
 'deriv1_encod_rel_rougher.input.feed_fe',
 'deriv1_encod_rel_rougher.input.feed_zn',
 'deriv1_encod_val_rougher.input.feed_fe',
 'deriv1_encod_val_rougher.input.feed_pb',
 'deriv1_encod_val_rougher.input.feed_zn',
 'deriv1_primary_cleaner.input.copper_sulfate',
 'deriv1_primary_cleaner.input.depressant',
 'deriv1_primary_cleaner.input.xanthate',
 'deriv1_primary_cleaner.state.floatbank8_c_air',
 'deriv1_primary_cleaner.state.floatbank8_c_level',
 'deriv1_primary_cleaner.state.floatbank8_d_level',
 'deriv1_rougher.input.feed_fe',
 'deriv1_rougher.input.feed_pb',
 'deriv1_rougher.input.feed_rate',
 'deriv1_rougher.input.feed_size',
 'deriv1_rougher.input.feed_sol',
 'deriv1_rougher.input.feed_zn',
 'deriv1_rougher.input.floatbank10_copper_sulfate',
 'deriv1_rougher.input.floatbank10_xanthate',
 'deriv1_rougher.input.floatbank11_copper_sulfate',
 'deriv1_rougher.input.floatbank11_xanthate',
 'de

# Base models

In [25]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.005, random_state=1,max_iter = 2000))

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.01, l1_ratio=.9, random_state=3))

CatBoost = CatBoostRegressor(loss_function='MAE',random_seed =123,learning_rate=0.1,max_depth=8,task_type='GPU',od_type = 'Iter',od_wait= 15,iterations = 2000)
model_xgb = xgb.XGBRegressor(learning_rate=0.05,
                            n_estimators=400,**{'max_depth': 8, 'gamma': '6.915', 'colsample_bytree': '0.501', 'min_child_weight': '5.338'},
                            silent=1,
                             random_state =7, nthread = -1)

model_lgb = lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.1, n_estimators=70,random_state=9,
                              **{'max_depth': 6, 'num_leaves': 6, 'feature_fraction': '0.650', 'bagging_fraction': '0.291'})
rf = RandomForestRegressor(n_estimators=120,max_depth = 5,max_features='auto',n_jobs= -1,criterion = 'mse')
#cv_oof(rf,X,y)

In [26]:
n_folds = 5


def cv_boost(model,X_base=None,y=None):
    cv = KFold(n_folds, shuffle=False, random_state=42)
    scores = []
    preds_all_alt = np.empty_like(y)
    preds_all_base = np.empty_like(y)
    
    true_all =np.empty_like(y)

    for fold_n, (train_index, valid_index) in enumerate(cv.split(X)):
    # print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        
        #print(y.shape)
        # Do the base
        evalset = [(X_train.values,y_train.values.reshape(-1,)),(X_valid.values, y_valid.values.reshape(-1,))]
        model.fit(X_train.values,y_train.values.reshape(-1,),eval_set=evalset,early_stopping_rounds = 20,verbose = False)
        
        if (model.__class__ == xgb.sklearn.XGBRegressor):
            print(model.best_ntree_limit)
            ntree = model.best_ntree_limit
            preds  = model.predict(X_valid.values, ntree_limit=ntree )
        
        if (model.__class__ == lgb.sklearn.LGBMRegressor):
            print(model.best_iteration)
            ntree = model.best_iteration_
            preds  = model.predict(X_valid.values, num_iterations=ntree)
        
        
        preds_all_base[valid_index] = preds
        true_all[valid_index] = y_valid
        
    oof_scores = mase(preds_all_base,true_all)
    return oof_scores



def cv_lgboost(model,X_base=None,y=None):
    cv = KFold(n_folds, shuffle=False, random_state=42)
    scores = []
    preds_all_alt = np.empty_like(y)
    preds_all_base = np.empty_like(y)
    
    true_all =np.empty_like(y)

    for fold_n, (train_index, valid_index) in enumerate(cv.split(X)):
    # print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        
        #print(y.shape)
        # Do the base
        evalset = [(X_valid.values, y_valid.values.reshape(-1,))]
        model.fit(X_train.values,y_train.values.reshape(-1,),eval_set=evalset,early_stopping_rounds = 20,verbose = False)
        
        ntree = model.best_iteration_
        preds  = model.predict(X_valid.values, num_iterations=ntree)
        
        
        preds_all_base[valid_index] = preds
        true_all[valid_index] = y_valid
        
    oof_scores = mase(preds_all_base,true_all)
    return oof_scores



def cv_oof(model,X_base=None,y=None):
    cv = KFold(n_folds, shuffle=False, random_state=42)
    preds = cross_val_predict(model,X=X_base,y=y,cv=cv,n_jobs = 1,method = 'predict')
    oof_scores = mase(preds,y)
    return oof_scores

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=False, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                
                instance.fit(np.array(X)[train_index], np.array(y)[train_index])
                y_pred = instance.predict(np.array(X)[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)
    
#cv_oof(model_xgb,X_filt,y)

# Try hyperopt for XGBoost

# Try hyperopt for LGBM

In [27]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

def objective(params):
    params = {
        #'max_depth': int(params['max_depth']),
        'num_leaves': int(params['num_leaves']), #int(max(2**(int(params['max_depth'])) - params['num_leaves'],0)),
        'feature_fraction': "{:.3f}".format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction']),
        'lambda_l1':params['lambda_l1']
        #"min_data_in_leaf": int(params['min_data_in_leaf'])

    }
    
    m = lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=500,random_state=9, 
                              **params)
    score = cv_lgboost(m,X_base = X_filt,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
   # 'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'num_leaves': hp.quniform('num_leaves', 8, 50, 1 ),
    'feature_fraction': hp.uniform('feature_fraction', 0.1, 0.9),
    'bagging_fraction':hp.uniform('bagging_fraction', 0.1, 1.0),
    'lambda_l1':hp.uniform('lambda_l1',0.1,40)
    #"min_data_in_leaf": hp.loguniform("min_data_in_leaf",1,8)
}

best_lgbm = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150)
# For Rougher, tsclean
# Score 2.465 params {'num_leaves': 57, 'feature_fraction': '0.048', 'bagging_fraction': '0.170'}
# Score 2.484 params {'num_leaves': 59, 'feature_fraction': '0.035', 'bagging_fraction': '0.411'}
# Score 2.495 params {'num_leaves': 10, 'feature_fraction': '0.079', 'bagging_fraction': '0.532'}
# Score 2.463 params {'num_leaves': 48, 'feature_fraction': '0.047', 'bagging_fraction': '0.515'}
# Score 2.407 params {'num_leaves': 11, 'feature_fraction': '0.057', 'bagging_fraction': '0.113'}
# Score 2.437 params {'num_leaves': 33, 'feature_fraction': '0.062', 'bagging_fraction': '0.102'}
# With Lambda l1 , tsclean
#Score 2.494 params {'num_leaves': 34, 'feature_fraction': '0.102', 'bagging_fraction': '0.372', 'lambda_l1': 10.48534583376155}
# Score 2.504 params {'num_leaves': 41, 'feature_fraction': '0.114', 'bagging_fraction': '0.386', 'lambda_l1': 12.546615754289613}
# Score 2.497 params {'num_leaves': 44, 'feature_fraction': '0.121', 'bagging_fraction': '0.175', 'lambda_l1': 5.356841244092816}
# Score 2.489 params {'num_leaves': 35, 'feature_fraction': '0.102', 'bagging_fraction': '0.818', 'lambda_l1': 20.3498175903831}


# For final, tsclean:

# Score 1.559 params {'num_leaves': 18, 'feature_fraction': '0.703', 'bagging_fraction': '0.535', 'lambda_l1': 6.7996621167358064}
# Score 1.559 params {'num_leaves': 27, 'feature_fraction': '0.549', 'bagging_fraction': '0.206', 'lambda_l1': 1.3634928026875381}
# Score 1.550 params {'num_leaves': 43, 'feature_fraction': '0.664', 'bagging_fraction': '0.083', 'lambda_l1': 2.845539961954432}
# Score 1.558 params {'num_leaves': 32, 'feature_fraction': '0.190', 'bagging_fraction': '0.340', 'lambda_l1': 1.504006451937044}
# Score 1.547 params {'num_leaves': 28, 'feature_fraction': '0.548', 'bagging_fraction': '0.270', 'lambda_l1': 0.2809097693267437}
# Score 1.573 params {'num_leaves': 27, 'feature_fraction': '0.448', 'bagging_fraction': '0.314', 'lambda_l1': 4.64151267093926}
# Score 1.541 params {'num_leaves': 15, 'feature_fraction': '0.439', 'bagging_fraction': '0.637', 'lambda_l1': 6.530043504380075}
# Score 1.545 params {'num_leaves': 40, 'feature_fraction': '0.709', 'bagging_fraction': '0.683', 'lambda_l1': 3.640592310230177}
# Score 1.550 params {'num_leaves': 33, 'feature_fraction': '0.586', 'bagging_fraction': '0.442', 'lambda_l1': 7.968821945221622}
# Score 1.551 params {'num_leaves': 23, 'feature_fraction': '0.777', 'bagging_fraction': '0.395', 'lambda_l1': 13.389210617182485}
# Score 1.557 params {'num_leaves': 18, 'feature_fraction': '0.503', 'bagging_fraction': '0.498', 'lambda_l1': 8.112492754813404}




Score 2.593 params {'num_leaves': 26, 'feature_fraction': '0.222', 'bagging_fraction': '0.888', 'lambda_l1': 3.5819564884805386}
Score 2.752 params {'num_leaves': 27, 'feature_fraction': '0.808', 'bagging_fraction': '0.768', 'lambda_l1': 39.022731836716964}
Score 2.560 params {'num_leaves': 27, 'feature_fraction': '0.186', 'bagging_fraction': '0.851', 'lambda_l1': 17.90583446846282}
Score 2.722 params {'num_leaves': 9, 'feature_fraction': '0.711', 'bagging_fraction': '0.977', 'lambda_l1': 2.150324482963201}
Score 2.772 params {'num_leaves': 50, 'feature_fraction': '0.879', 'bagging_fraction': '0.856', 'lambda_l1': 21.10755845033811}
Score 2.629 params {'num_leaves': 11, 'feature_fraction': '0.249', 'bagging_fraction': '0.859', 'lambda_l1': 21.963760955647555}
Score 2.774 params {'num_leaves': 39, 'feature_fraction': '0.888', 'bagging_fraction': '0.512', 'lambda_l1': 38.748126283885625}
Score 2.659 params {'num_leaves': 20, 'feature_fraction': '0.337', 'bagging_fraction': '0.832', 'lamb

Score 2.648 params {'num_leaves': 29, 'feature_fraction': '0.400', 'bagging_fraction': '0.621', 'lambda_l1': 37.07019395267663}
Score 2.622 params {'num_leaves': 45, 'feature_fraction': '0.258', 'bagging_fraction': '0.130', 'lambda_l1': 4.411859550299761}
Score 2.564 params {'num_leaves': 33, 'feature_fraction': '0.138', 'bagging_fraction': '0.569', 'lambda_l1': 15.005107656009292}
Score 2.489 params {'num_leaves': 35, 'feature_fraction': '0.102', 'bagging_fraction': '0.818', 'lambda_l1': 20.3498175903831}
Score 2.492 params {'num_leaves': 32, 'feature_fraction': '0.104', 'bagging_fraction': '0.873', 'lambda_l1': 17.960911169114063}
Score 2.619 params {'num_leaves': 27, 'feature_fraction': '0.243', 'bagging_fraction': '0.993', 'lambda_l1': 20.106460237529745}
Score 2.492 params {'num_leaves': 32, 'feature_fraction': '0.103', 'bagging_fraction': '0.897', 'lambda_l1': 17.649752786345843}
Score 2.588 params {'num_leaves': 32, 'feature_fraction': '0.203', 'bagging_fraction': '0.935', 'lamb

KeyboardInterrupt: 

In [8]:
from rgf.sklearn import RGFRegressor
from hyperopt import hp, tpe
from hyperopt.fmin import fmin


def cv_rgf(model,X_base=None,y=None):
    cv = KFold(n_folds, shuffle=False, random_state=42)
    scores = []
    preds_all_alt = np.empty_like(y)
    preds_all_base = np.empty_like(y)
    
    true_all =np.empty_like(y)

    for fold_n, (train_index, valid_index) in enumerate(cv.split(X)):
    # print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        
        #print(y.shape)
        # Do the base
        model.fit(X_train.values,y_train.values.reshape(-1,))
        
        preds  = model.predict(X_valid.values)
        
        
        preds_all_base[valid_index] = preds
        true_all[valid_index] = y_valid
        
    oof_scores = mase(preds_all_base,true_all)
    return oof_scores


def objective(params):
    params = {
        'test_interval': int(params['test_interval'])
        #'max_depth': int(params['max_depth']),
       # 'num_leaves': int(params['num_leaves']), #int(max(2**(int(params['max_depth'])) - params['num_leaves'],0)),
        #'feature_fraction': "{:.3f}".format(params['feature_fraction']),
       # 'bagging_fraction': '{:.3f}'.format(params['bagging_fraction'])
        #"min_data_in_leaf": int(params['min_data_in_leaf'])

    }
    
    m =RGFRegressor(max_leaf=400,
                    algorithm="RGF",
                    #test_interval=100,
                    loss= 'LS',
                    normalize =True,
                    verbose=True,**params)
    score = cv_rgf(m,X_base = X,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    #'max_leaf': hp.loguniform('num_leaves', 2, 8),
    #'test_interval': hp.quniform('test_interval', 100, 1000,100),
    #'bagging_fraction':hp.uniform('bagging_fraction', 0.05, 1.0),
    #"min_data_in_leaf": hp.loguniform("min_data_in_leaf",1,8)
}

best_rgf = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150)



"train": 
   algorithm=RGF
   train_x_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\561bd4d6-2c45-47de-b11d-37f6ec82b7651.train.data.x
   train_y_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\561bd4d6-2c45-47de-b11d-37f6ec82b7651.train.data.y
   Log:ON
   model_fn_prefix=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\561bd4d6-2c45-47de-b11d-37f6ec82b7651.model
--------------------
Thu Feb 28 16:42:45 2019: Reading training data ... 
Thu Feb 28 16:42:46 2019: Start ... #train=10872
--------------------
Forest-level: 
   loss=LS
   max_leaf_forest=400
   max_tree=200
   opt_interval=100
   test_interval=300
   num_tree_search=1
   Verbose:ON
   memory_policy=Generous
-------------
Training data: 343x10872, nonzero_ratio=0.9963; managed as dense data.
-------------
Optimization: 
   loss=LS
   num_iteration_opt=10
   reg_L2=0.1
   opt_stepsize=0.5
   NormalizeTarget:ON
Tree-level: min_pop=10
Node split: reg_L2=0.1
--------------------
Thu Feb 28 16:42:56 2019: Calling optimizer with 32 trees and 10


"predict": 
   model_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\6e430655-11d9-4f53-9fe6-d956198c7ab54.model-02
   test_x_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\6e430655-11d9-4f53-9fe6-d956198c7ab54.test.data.x
   prediction_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\6e430655-11d9-4f53-9fe6-d956198c7ab54.predictions.txt
   Log:ON
--------------------
Thu Feb 28 16:45:36 2019: Reading test data ... 
Thu Feb 28 16:45:36 2019: Predicting ... 
elapsed: 0.011
C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\6e430655-11d9-4f53-9fe6-d956198c7ab54.predictions.txt: C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\6e430655-11d9-4f53-9fe6-d956198c7ab54.model-02,#leaf=400,#tree=90
Thu Feb 28 16:45:36 2019: Done ... 

"train": 
   algorithm=RGF
   train_x_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\79e2dbc9-056b-4491-8565-56faed9ed5385.train.data.x
   train_y_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\79e2dbc9-056b-4491-8565-56faed9ed5385.train.data.y
   Log:ON
   model_fn_prefix=C:\Users\ABIRYU~1\AppData\


"predict": 
   model_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\73565be3-5ad4-4d56-bda6-0c691bee025c8.model-01
   test_x_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\73565be3-5ad4-4d56-bda6-0c691bee025c8.test.data.x
   prediction_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\73565be3-5ad4-4d56-bda6-0c691bee025c8.predictions.txt
   Log:ON
--------------------
Thu Feb 28 16:48:28 2019: Reading test data ... 
Thu Feb 28 16:48:29 2019: Predicting ... 
elapsed: 0.011
C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\73565be3-5ad4-4d56-bda6-0c691bee025c8.predictions.txt: C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\73565be3-5ad4-4d56-bda6-0c691bee025c8.model-01,#leaf=400,#tree=92
Thu Feb 28 16:48:29 2019: Done ... 

"train": 
   algorithm=RGF
   train_x_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\a66233b7-21a8-4bae-a76a-8095255985dd9.train.data.x
   train_y_fn=C:\Users\ABIRYU~1\AppData\Local\Temp\rgf\a66233b7-21a8-4bae-a76a-8095255985dd9.train.data.y
   Log:ON
   model_fn_prefix=C:\Users\ABIRYU~1\AppData\

KeyboardInterrupt: 

In [ ]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'subsample':"{:.3f}".format(params['subsample']),
        'min_child_weight':"{:.3f}".format(params['min_child_weight'])

    }
    # Score 1.226 params {'max_depth': 2, 'gamma': '0.544', 'colsample_bytree': '0.684', 'subsample': '0.932'}

    m = xgb.XGBRegressor(learning_rate=0.05,  
                             n_estimators=300,                        
                             random_state =123,verbose=0,silent=True, n_jobs = -1,**params)
    score = cv_boost(m,X_base = X_filt,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3,1.0),
    'min_child_weight':hp.loguniform('min_child_weight', -3, 5),
    'subsample':hp.uniform('subsample', 0.3,1.0),
    'gamma': hp.uniform('gamma',5, 100.)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=250)
#depth 9 gam 7.6 col 0.5


# For final!
# X_filt
# Score 1.693 params {'max_depth': 4, 'gamma': '9.397', 'colsample_bytree': '0.496', 'min_child_weight': '4.803'}
# Score 1.711 params {'max_depth': 4, 'gamma': '2.640', 'colsample_bytree': '0.461', 'min_child_weight': '1.456'}
# Score 1.698 params {'max_depth': 4, 'gamma': '7.779', 'colsample_bytree': '0.570', 'min_child_weight': '1.442'}
# Score 1.703 params {'max_depth': 2, 'gamma': '9.923', 'colsample_bytree': '0.510', 'subsample': '0.425'}
# Score 1.704 params {'max_depth': 2, 'gamma': '4.949', 'colsample_bytree': '0.532', 'subsample': '0.868', 'min_child_weight': '6.246'}
# Score 1.713 params {'max_depth': 4, 'gamma': '1.359', 'colsample_bytree': '0.380', 'subsample': '0.657', 'min_child_weight': '8.415'}
# Score 1.718 params {'max_depth': 3, 'gamma': '3.953', 'colsample_bytree': '0.595', 'subsample': '0.688', 'min_child_weight': '9.466'}
# Score 1.677 params {'max_depth': 5, 'gamma': '11.269', 'colsample_bytree': '0.452', 'subsample': '0.577', 'min_child_weight': '17.510'}
# Score 1.698 params {'max_depth': 3, 'gamma': '7.361', 'colsample_bytree': '0.493', 'subsample': '0.510', 'min_child_weight': '3.411'}
#Lagged X
# Score 1.590 params {'max_depth': 7, 'gamma': '84.207', 'colsample_bytree': '0.910', 'subsample': '0.779', 'min_child_weight': '80.419'}
# Score 1.598 params {'max_depth': 3, 'gamma': '91.892', 'colsample_bytree': '0.468', 'subsample': '0.337', 'min_child_weight': '56.616'}
# Score 1.598 params {'max_depth': 5, 'gamma': '74.885', 'colsample_bytree': '0.450', 'subsample': '0.534', 'min_child_weight': '21.323'}
# Score 1.606 params {'max_depth': 5, 'gamma': '70.908', 'colsample_bytree': '0.585', 'subsample': '0.681', 'min_child_weight': '49.926'}
# Score 1.609 params {'max_depth': 6, 'gamma': '30.216', 'colsample_bytree': '0.773', 'subsample': '0.804', 'min_child_weight': '65.349'}
# Score 1.591 params {'max_depth': 6, 'gamma': '50.306', 'colsample_bytree': '0.736', 'subsample': '0.945', 'min_child_weight': '64.534'}



# For rougher

# Score 2.611 params {'max_depth': 6, 'gamma': '16.247', 'colsample_bytree': '0.377', 'subsample': '0.694', 'min_child_weight': '12.482'}
# Score 2.655 params {'max_depth': 4, 'gamma': '8.024', 'colsample_bytree': '0.363', 'subsample': '0.708', 'min_child_weight': '9.340'}
# Score 2.666 params {'max_depth': 4, 'gamma': '16.597', 'colsample_bytree': '0.434', 'subsample': '0.866', 'min_child_weight': '10.709'}
# Score 2.615 params {'max_depth': 5, 'gamma': '3.906', 'colsample_bytree': '0.401', 'subsample': '0.760', 'min_child_weight': '9.563'}
# Score 2.608 params {'max_depth': 4, 'gamma': '5.102', 'colsample_bytree': '0.344', 'subsample': '0.884', 'min_child_weight': '9.622'}
# Score 2.608 params {'max_depth': 6, 'gamma': '18.904', 'colsample_bytree': '0.412', 'subsample': '0.666', 'min_child_weight': '13.043'}
# Score 2.603 params {'max_depth': 8, 'gamma': '26.395', 'colsample_bytree': '0.332', 'subsample': '0.752', 'min_child_weight': '19.750'}
# Score 2.580 params {'max_depth': 9, 'gamma': '23.565', 'colsample_bytree': '0.536', 'subsample': '0.935', 'min_child_weight': '17.580'}
# Score 2.608 params {'max_depth': 8, 'gamma': '22.799', 'colsample_bytree': '0.388', 'subsample': '0.697', 'min_child_weight': '10.327'}
# Score 2.599 params {'max_depth': 7, 'gamma': '44.954', 'colsample_bytree': '0.395', 'subsample': '0.993', 'min_child_weight': '133.132'}
# 



In [ ]:
X

In [ ]:
m = LGBMRegressor()
#Xfilt lagged, one lag:
# Score 2.480 params {'max_depth': 5, 'num_leaves': 40, 'feature_fraction': '0.453', 'bagging_fraction': '0.276'}
# Score 2.500 params {'max_depth': 4, 'num_leaves': 180, 'feature_fraction': '0.322', 'bagging_fraction': '0.150'}
# Score 2.490 params {'max_depth': 5, 'num_leaves': 80, 'feature_fraction': '0.440', 'bagging_fraction': '0.666'}
# Score 2.490 params {'max_depth': 6, 'num_leaves': 25, 'feature_fraction': '0.555', 'bagging_fraction': '0.257'}
# Score 2.484 params {'max_depth': 4, 'num_leaves': 30, 'feature_fraction': '0.658', 'bagging_fraction': '0.863'}


# Try HyperOpt for SVR

In [28]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
scaler = make_pipeline(QuantileTransformer(output_distribution='normal'),StandardScaler(),PCA(whiten=True))
X_sc = scaler.fit_transform(X)
def objective(params):
    params = {
        'C': params['C'],
      #  'epsilon':"{:.3f}".format(params['num_leaves']),
        'gamma': params['gamma']
    }
    m = SVR(kernel = 'rbf',**params)
    score = cv_oof(m,X_base = Xsc,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    'C': hp.loguniform('C', -5, 5),
    'gamma': hp.loguniform('gamma', -2,4 )
}

best_lgbm = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150)
#Score 2.560 params {'C': 15.130365120277071, 'gamma': 51.040699174724814}


NameError: name 'QuantileTransformer' is not defined

# Try HyperOpt for Catboost

In [ ]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'l2_leaf_reg': params['l2_leaf_reg'],
       # 'colsample_bylevel': params['colsample_bylevel'],
     #   'random_strength':params['random_strength'],
        #'verbose_eval':1000
    }
    # Score 1.226 params {'max_depth': 2, 'gamma': '0.544', 'colsample_bytree': '0.684', 'subsample': '0.932'}
    m = CatBoostRegressor(loss_function='Quantile:alpha=0.6',random_seed =123,learning_rate=0.4,task_type='GPU',od_type = 'Iter',od_wait= 15,iterations = 1000,
                          max_bin=128,**params)

    score = cv_catboost(m,X_base = X_filt,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    'max_depth': hp.quniform('max_depth', 3, 5, 1),
    #'colsample_bylevel': hp.uniform('colsample_bylevel', 0.3, 1.0),
    'l2_leaf_reg': hp.loguniform('l2_leaf_reg', -8, 1),
    #'random_strength':hp.loguniform('random_strength', -5, 5)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150)

# Try hyperopt KNN

In [ ]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.decomposition import PCA

scaler = make_pipeline(QuantileTransformer(output_distribution='normal'),StandardScaler(),PCA(whiten=True))
X_sc = scaler.fit_transform(X)
    


def objective(params):
    params = {
        'n_neighbors': int(params['n_neighbors']),
        'weights':params['weights'],
        'leaf_size': params['leaf_size']
    }
    
    
    m = KNeighborsRegressor(n_jobs = -1,**params)
    score = cv_oof(m,X_base = X_sc,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    'n_neighbors': hp.quniform('n_neighbors',3, 256,2),
    'leaf_size': hp.quniform('lef_size', 14,50,2 ),
    'weights':hp.choice('weights', ['uniform','distance'])
}

best_lgbm = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150)
# Score 3.224 params {'n_neighbors': 198, 'weights': 'distance', 'leaf_size': 36.0}
# Score 3.194 params {'n_neighbors': 254, 'weights': 'distance', 'leaf_size': 16.0}
# Score 3.196 params {'n_neighbors': 250, 'weights': 'distance', 'leaf_size': 18.0}
# Score 3.202 params {'n_neighbors': 238, 'weights': 'distance', 'leaf_size': 48.0}
# Score 3.215 params {'n_neighbors': 214, 'weights': 'distance', 'leaf_size': 30.0}



In [ ]:
y.plot(figsize  = (20,5))

In [ ]:
y.apply(pd.isna,axis=0).sum()

In [ ]:
diffs.plot()

In [ ]:
y[tgt].values